# Análsis de sentimiento de noticias financieras con FinBERT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1-OJkcwMAcLm2S5A1glyLhyQH6bbaq5F2)

## Tabla de contenido

1. Obtención y preparación de los datos
2. Análisis de sentimiento con `FinBERT`
3. Creación de una app con `Streamlit` (Ver app.py)

### Imports

In [1]:
import torch
from torch.utils.data import DataLoader
from torch import cuda

import transformers

import numpy as np
import pandas as pd

import feedparser
from tqdm import tqdm

## 1. Obtención y preparación de los datos

La fuente de noticias elegida en este caso es el medio [Seeking Alpha](https://seekingalpha.com). Seeking Alpha ofrece feeds de noticias gratuitas via RSS (Really Simple Syndication), que es un formato de texto estándar y público que sirve para distribuir información de forma automatizada.

Para parsear las noticias, podemos utilizar el modulo de python `feedparser`, que de forma simple y sencilla nos permite obtener estas feed de noticias en formato RSS.

Para esta tarea, creamos una función llamada `rss_parser()`. Esta función toma como parámetro la url de la fuente de la que queremos extraer las noticias, utiliza el metodo `parse()` del módulo `feedparser` para parsear las noticias desde la url y devuelve la siguiente información en un diccionario de python:
- Titulo
- Enlace a la noticia
- Fecha de publicación
- Nombre de la empresa
- Nombre del autor de la noticia

In [2]:
def rss_parser(url):
    """
    Seeking Alpha RSS news parser.

    Parameters
    ----------
    url: str, rss feed url

    Returns
    -------
    news: dict, news feed by seeking alpha
    """
    # parse news with feedparser
    try:
        NewsFeed = feedparser.parse(url)
    except Exception as e:
        print("RSS feed not found.")
        print(e)

    # get entries
    entries = NewsFeed.entries
    n_entries = len(entries)
    if n_entries < 1:
        print("Ticker not found or no news available.")
        return None
    print(f"Number of RSS posts: {n_entries}")

    # dict keys
    keys = ["title", "link", "published", "sa_company_name", "sa_author_name"]

    # store news data in dict
    news = dict()
    for idx in range(n_entries):
        # get entry
        entry = entries[idx]
        # save relevant data in dict
        entry_dict = {key: entry[key] for key in keys}
        # save news
        news[idx] = entry_dict
    
    return news

> Ejemplo del funcionamiento de la función `rss_parser()`

In [3]:
# seeking alpha RSS feed by stock
ticker = "TSLA"
url = f"https://seekingalpha.com/api/sa/combined/{ticker}.xml"
news = rss_parser(url)
print(news)
print(type(news))

Number of RSS posts: 30
{0: {'title': "Renewed Focus On Tesla's Energy Storage Growth Amidst New Macro Developments", 'link': 'https://seekingalpha.com/article/4456915-renewed-focus-on-teslas-energy-storage?source=feed_symbol_TSLA', 'published': 'Fri, 24 Sep 2021 09:46:56 -0400', 'sa_company_name': 'Tesla, Inc.', 'sa_author_name': 'Nick Cox'}, 1: {'title': "Elon Musk upgrades global chip shortage to just a 'short-term' problem", 'link': 'https://seekingalpha.com/symbol/TSLA/news?source=feed_symbol_TSLA', 'published': 'Fri, 24 Sep 2021 08:46:22 -0400', 'sa_company_name': 'Faraday Future Intelligent Electric Inc.', 'sa_author_name': 'Clark Schultz'}, 2: {'title': 'Tesla: A Hedge Against Inflation', 'link': 'https://seekingalpha.com/article/4456771-tesla-tsla-stock-a-hedge-against-inflation?source=feed_symbol_TSLA', 'published': 'Thu, 23 Sep 2021 13:19:18 -0400', 'sa_company_name': 'Tesla, Inc.', 'sa_author_name': 'Real Investments'}, 3: {'title': 'Tesla Sees Some Speed Bumps In A Quest T

In [4]:
# visualise in rectangular format
pd.DataFrame(news).T.head(10)

,title,link,published,sa_company_name,sa_author_name
0,Renewed Focus On Tesla's Energy Storage Growth...,https://seekingalpha.com/article/4456915-renew...,"Fri, 24 Sep 2021 09:46:56 -0400","Tesla, Inc.",Nick Cox
1,Elon Musk upgrades global chip shortage to jus...,https://seekingalpha.com/symbol/TSLA/news?sour...,"Fri, 24 Sep 2021 08:46:22 -0400",Faraday Future Intelligent Electric Inc.,Clark Schultz
2,Tesla: A Hedge Against Inflation,https://seekingalpha.com/article/4456771-tesla...,"Thu, 23 Sep 2021 13:19:18 -0400","Tesla, Inc.",Real Investments
3,Tesla Sees Some Speed Bumps In A Quest To Win ...,https://seekingalpha.com/article/4456761-tesla...,"Thu, 23 Sep 2021 12:32:13 -0400","Tesla, Inc.",Damien Robbins
4,"Tesla sees a new bear on the prowl with Tudor,...",https://seekingalpha.com/symbol/TSLA/news?sour...,"Thu, 23 Sep 2021 08:47:46 -0400","Tesla, Inc.",Clark Schultz
5,Tesla Stock Forecast: Is Their EV Leadership F...,https://seekingalpha.com/article/4456426-tesla...,"Tue, 21 Sep 2021 17:04:50 -0400","Tesla, Inc.",JR Research
6,Tesla: The Real Gamma Squeeze,https://seekingalpha.com/article/4456217-tesla...,"Tue, 21 Sep 2021 09:30:00 -0400","Tesla, Inc.",Ahan Vashi
7,Patent watch: Tesla lands approval for cooling...,https://seekingalpha.com/symbol/TSLA/news?sour...,"Tue, 21 Sep 2021 07:00:52 -0400","Rivian Automotive, LLC",Clark Schultz
8,Tesla draws extra scrutiny with NTSB investiga...,https://seekingalpha.com/symbol/TSLA/news?sour...,"Sat, 18 Sep 2021 07:00:42 -0400","Tesla, Inc.",Clark Schultz
9,Tesla forecast to see 1.3M deliveries in 2022 ...,https://seekingalpha.com/symbol/TSLA/news?sour...,"Fri, 17 Sep 2021 07:41:14 -0400","Tesla, Inc.",Clark Schultz


Para analizar el sentimiento sobre un determinado ticker o valor vamos a utilizar un modelo BERT pre-entrenado en un corpus financiero. Utilizaremos el framework de deep learning Pytorch.

En primer lugar, vamos a crear una celda que denominaremos `config`, donde crearemos variables que contendras objetos/valores que usaremos durante el procesado de los datos y modelización.

### Config

In [5]:
# max number of tokens
MAX_LEN = 512
BATCH_SIZE = 64
# model path (on huggingface)
FINBERT_PATH = "ProsusAI/finbert"
# load model tokenizer
TOKENIZER = transformers.BertTokenizer.from_pretrained("ProsusAI/finbert")

# dataloader params
PARAMS = {'batch_size': BATCH_SIZE,
          'shuffle': False,
          'num_workers': 4
        }

# Setting up the device for GPU usage if available
DEVICE = 'cuda' if cuda.is_available() else 'cpu'

Una vez hemos definido estas variables que iremos utilizando a lo largo de todo el procesa, necesitamos crear un objeto dataset the Pytorch. En este caso, vamos a utilizar un dataset del tipo [map-iterable](https://pytorch.org/docs/stable/data.html).

In [13]:
class FinBERTDataset:
    """
    Map-iterable Pytorch dataset for FinBERT.

    Arguments:
    ---------
    title: list, news titles
    target: int, only for training and evaluation
    """

    def __init__(self, title, target=None):
        self.title = title
        self.target = target    # list of number: 0 or 1
        self.tokenizer = TOKENIZER  # FinBERT tokenizer
        self.max_len = MAX_LEN  # Max number of tokens

    def __len__(self):
        return len(self.title)

    def __getitem__(self, item):  # takes a item and returns your items in the dataset
        title = str(self.title[item])
        title = " ".join(title.split())
        # prepare data as per FinBERT requirements
        inputs = self.tokenizer.encode_plus(
            title,
            None,   # second string is None in this case
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        # if training or evalution, return targets as well
        if self.target:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'target': torch.tensor(self.target[item], dtype=torch.float)
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            }

> Ejemplo de funcionamiento de una map-iterable dataset de Pytorch

In [ ]:
# convert json to list of news
news_list = [news[k]["title"] for k in news.keys()]
# create torch dataset and dataloader
dataset = FinBERTDataset(news_list)
# visualise one example in dataset
print(dataset[1])

{'ids': tensor([  101,  3449,  2239, 14163,  6711, 18739,  3795,  9090, 15843,  2000,
         2074,  1037,  1005,  2460,  1011,  2744,  1005,  3291,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,

Almacenaremos las noticias extraídas del feed RSS en este dataset y cargaremos este dataset en un DataLoader de Pytorch, que no es más que un generator de python.

## 2. Análisis de sentimiento

Para analizar el sentimiento de noticias vamos a utilizar el modelo pre-entrenado en un corpus financiero FinBERT. Este modelo fue publicado por ProsusAI en el paper [FinBERT: Financial Sentiment Analysis with Pre-trained Language Models](https://arxiv.org/abs/1908.10063), y nos permite realizar un análsis de sentimiento en noticias financieras de forma muy rápida y sin necesidad de hacer un pre-entrenamiento o fine-tuning.

Para obtener el sentimiento de las noticias, vamos a crear una función llamada `run_inference()` que a un alto nivel realiza lo siguiente:
1. Toma como parámetro un archivo json con las noticias extraídas del feed RSS
2. Almacena las noticias en el map-iterable dataset que preparamos y carga este dataset en un DataLoader para pasar los datos en lotes
3. Descarga el modelo FinBERT del hub de modelos de Hugginface y lo inicializa para una tarea de clasificación
4. Realiza el análisis de sentimiento y almacena los resultados en una lista
5. Utiliza la función `decode_sentiment()` para mapear valores números a sentimiento {0: "Positive", 1: "Negative", 2: "Neutral"} 

In [14]:
def decode_sentiment(inferences):
    """Decode sentiment."""
    decoded_sentiment = []
    for sentiment in inferences:
        if sentiment == 0:
            decoded_sentiment.append("Positive")
        elif sentiment == 1:
            decoded_sentiment.append("Negative")
        else:
            decoded_sentiment.append("Neutral")

    return decoded_sentiment


def run_inference(news):
    """
    Runs inference on data.

    Parameters
    ----------
    news: json file, news from RSS feed

    Returns
    -------
    inferences: list, sentiment for each title
    """
    # convert json to list of news
    news_list = [news[k]["title"] for k in news.keys()]
    # create torch dataset and dataloader
    inference_set = FinBERTDataset(news_list)
    data_loader = DataLoader(inference_set, **PARAMS)
    # load model
    model = transformers.BertForSequenceClassification.from_pretrained(FINBERT_PATH)
    model.eval()
    device = torch.device(DEVICE)
    model.to(device)
    # inference
    outputs_list = []
    with torch.no_grad():
        for _, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d['ids']
            mask = d['mask']
            token_type_ids = d["token_type_ids"]


            # send them to the cuda device we are using
            ids = ids.to(torch.device(device), dtype=torch.long)
            mask = mask.to(torch.device(device), dtype=torch.long)
            token_type_ids = token_type_ids.to(torch.device(device), dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                token_type_ids=token_type_ids
            )

            outputs_list.extend(torch.softmax(outputs.logits, dim=1).cpu().detach().numpy().tolist())

    # get most probable class
    inferences = np.argmax(outputs_list, axis=1)

    # decode sentiment
    sentiments = decode_sentiment(inferences)

    return sentiments

> Ejemplo del funcionamiento de la función `run_inference()`

In [15]:
run_inference(news)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/sentiment_app/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/sentiment_app/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'FinBERTDataset' on <module '__main__' (built-in)>


KeyboardInterrupt: 

Utilizaremos esta función para obtener los sentimiento de las noticias.

## 3. Creación de una app con Streamlit

[Streamlit](https://streamlit.io/) es un framework de python para desarrollar en minutos web apps que nos permite compartir y visualizar resultados. Elimina la barrera de tener conocimientos de front-end y es muy utilizada por equipos de data science.

Vamos a utilizar Streamlit para desarollar una pequeña app de análisis de sentimiento de noticias. La app permitirá a el usuario introducir el ticker del valor del cual desea realizar el analisis de sentimiento en la noticias y podrá visualizar estas noticias con su respectivo sentimiento. Además, también podrá seguir el link a cada noticia para realizar un análisis más detallado de la noticia.

El script `app.py`, además de contener el código que permite crear la app con Streamlit, utiliza la función `rss_parser()` y `run_inference()` para realizar el analisis de sentimiento en el ticker especificado.

In [21]:
ticker = input("Ticker:")

Ticker:TSLA


In [23]:
# parse news
url = f"https://seekingalpha.com/api/sa/combined/{ticker}.xml"
news = rss_parser(url)
# sentiment -> call inference
sentiments = run_inference(news)
print(sentiments)

Number of RSS posts: 30


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1/1 [00:53<00:00, 53.55s/it]

['Negative', 'Positive', 'Neutral', 'Positive', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative']
